In [161]:
# Load Packages
import numpy as np
import pandas as pd

In [162]:
#Load dataset and create a dataframe
data_imgs = np.loadtxt('imgs-train.txt', delimiter = ' ')
data_label = np.loadtxt('label-train.txt')
data_label = data_label[:,1]
data_imgs = data_imgs.astype(np.float32)
data_label = data_label.astype(np.float32)
df_imgs = pd.DataFrame(data_imgs)
df_imgs
df_label = pd.DataFrame(data_label)
df_label
df = pd.concat([df_imgs, df_label], axis=1)

df.shape

(4546, 721)

In [163]:
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

#Split dataset into train set and test set
X = df.iloc[:,0:720]
y = df.iloc[:,720]
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X, y, test_size = 0.33, random_state = 1992)

In [164]:
X_train_raw.head()

,0,1,2,3,4,5,6,7,8,9,...,710,711,712,713,714,715,716,717,718,719
3966,1324.0,1284.0,1118.0,984.0,878.0,504.0,406.0,684.0,1082.0,1330.0,...,422.0,754.0,832.0,1142.0,958.0,1102.0,988.0,888.0,674.0,456.0
2749,260.0,190.0,202.0,132.0,260.0,386.0,452.0,350.0,360.0,620.0,...,742.0,732.0,834.0,758.0,874.0,734.0,572.0,426.0,446.0,464.0
2530,498.0,692.0,330.0,296.0,184.0,344.0,422.0,474.0,348.0,308.0,...,336.0,376.0,592.0,962.0,948.0,686.0,360.0,252.0,338.0,284.0
3065,392.0,386.0,144.0,120.0,78.0,98.0,80.0,70.0,22.0,64.0,...,138.0,94.0,176.0,296.0,320.0,212.0,142.0,98.0,154.0,316.0
1066,0.0,0.0,98.0,168.0,240.0,126.0,64.0,38.0,80.0,62.0,...,172.0,276.0,460.0,316.0,324.0,246.0,264.0,128.0,48.0,0.0


In [165]:
y_train_raw.head()

3966    1.0
2749    0.0
2530    1.0
3065    0.0
1066    1.0
Name: 0, dtype: float32

In [166]:
#Transfer dataframe to array and create a better shape
X_train = np.array(X_train_raw)
X_train_image= tf.reshape(X_train, [-1,60,12,1])

X_test = np.array(X_test_raw)
X_test_image= tf.reshape(X_test, [-1,60,12,1])

y_train = np.array(y_train_raw)
y_train = y_train.astype(np.float32).reshape(-1,1)
y_train = np.array(OneHotEncoder().fit_transform(y_train).todense())

y_test = np.array(y_test_raw)
y_test = y_test.astype(np.float32).reshape(-1,1)
y_test = np.array(OneHotEncoder().fit_transform(y_test).todense())


In [167]:
y_test

array([[ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [168]:
X_train.shape

(3045, 720)

In [169]:
x  = tf.placeholder("float", shape=[None, 720])
y_ = tf.placeholder("float", shape=[None, 2])

x_image = tf.reshape(x, [-1,60,12,1])

In [170]:
#Define 
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [218]:
W_conv1 = weight_variable([1, 1, 1, 8])
b_conv1 = bias_variable([8])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


W_conv2 = weight_variable([4, 4, 8, 12])
b_conv2 = bias_variable([12])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


W_fc1 = weight_variable([15 * 3 * 12, 24])
b_fc1 = bias_variable([24])

h_pool2_flat = tf.reshape(h_pool2, [-1, 15 * 3 * 12])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([24, 2])
b_fc2 = bias_variable([2])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [219]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [226]:
#Run and check the accuracy
batch_size = 20

for i in range(1000):
    indices_train = np.random.choice(X_train.shape[0], batch_size)
    images_train_batch = X_train[indices_train]
    labels_train_batch = y_train[indices_train]
    
    indices_test = np.random.choice(X_test.shape[0], batch_size)
    images_test_batch = X_test[indices_test]
    labels_test_batch = y_test[indices_test]
    if i%100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x: images_train_batch,
                                                           y_: labels_train_batch, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))

    sess.run(train_step,feed_dict={x: images_test_batch,y_: labels_test_batch, keep_prob: 0.5})

print("test accuracy %g"% sess.run(accuracy, feed_dict={x: images_test_batch,y_: labels_test_batch, keep_prob: 1.0}))



step 0, training accuracy 0.75
step 100, training accuracy 0.7
step 200, training accuracy 0.7
step 300, training accuracy 0.75
step 400, training accuracy 0.8
step 500, training accuracy 0.8
step 600, training accuracy 0.6
step 700, training accuracy 0.65
step 800, training accuracy 0.65
step 900, training accuracy 0.5
test accuracy 0.8
